![Rampart Logo](../images/logo.png)

Auge is an image classification model. Its main target's to recognize common photos to determine a few flat publication's features. Each image belongs to a specific realty, having a bunch of photo & panorama recognized **twinkle** can better predict apartments' order.

## I/O
Required data consists of two parts + remote image CDN:
- Training references lie in `../data/auge/training.csv` ;
- Testing links lie in `../data/auge/testing.csv` .

Currently the DB containes images of two types: simple photos & wide (360 deg) panoramas. Until a stable classification model panoramas should be omitted. Final classifier must be stored into `../models/auge.pth` .

## Classes
- `unknown` is the default class of an image tuple. Unclassified photos & broken/abandoned links also obtain this label.
- `luxury` is a flat with rich furniture, huge rooms, chandeliers, fireplaces, etc.
![Luxury 1](../images/luxury1.webp)
![Luxury 2](../images/luxury2.webp)
![Luxury 3](../images/luxury3.webp)
- `comfort` is the most suitable for an ordinary citizen apartments. Clean, neat, sometimes minimalistic, average area, qualitive furniture, etc.
![Comfort 1](../images/comfort1.webp)
![Comfort 2](../images/comfort2.webp)
![Comfort 3](../images/comfort3.webp)
- `construction` is a flat without a finished design. No doors, floor, supplies, wallpapers, ceiling, furniture, etc. Typically, new buildings contain these apartments.
![Construction 1](../images/construction1.webp)
![Construction 2](../images/construction2.webp)
![Construction 3](../images/construction3.webp)
- `junk` is an old flat image. Probably, the whole apartments should belong to a dormitory, Khrushchevka or gostinka.
![Junk 1](../images/junk1.webp)
![Junk 2](../images/junk2.webp)
![Junk 3](../images/junk3.webp)
- `obsolete` is an old house photo. Actually, it's a close neighbour of the previous label.
![Obsolete 1](../images/obsolete1.webp)
![Obsolete 2](../images/obsolete2.webp)
![Obsolete 3](../images/obsolete3.webp)
- `outline` is not a flat itself, it's just a drawing. Quite unpredictable label, but often these photos are attached to flats in new houses.
![Outline 1](../images/outline1.webp)
![Outline 2](../images/outline2.webp)
![Outline 3](../images/outline3.webp)
- `novelty` is an image of a house. What's more, it's a new building. But these photos have a trick: inner apartments may be under construction, but the image can't reflect it.
![Novelty 1](../images/novelty1.webp)
![Novelty 2](../images/novelty2.webp)
![Novelty 3](../images/novelty3.webp)
- `view` is a photo from the flat's balcony or window. Typically, includes sky, houses & yards nearby, sea, far fields & forests.
![View 1](../images/view1.webp)
![View 2](../images/view2.webp)
![View 3](../images/view3.webp)

In [ ]:
%matplotlib inline

In [ ]:
from cv2 import imread, IMREAD_COLOR, cvtColor, COLOR_BGR2RGB, IMREAD_GRAYSCALE, imwrite, inpaint, INPAINT_TELEA
from plotly.subplots import make_subplots
from plotly.graph_objs import Image, Pie, Figure
from plotly.offline import init_notebook_mode, iplot
from plotly.express import imshow
from re import match
from numpy import stack, copy
from glob import glob
from pandas import DataFrame
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Compose, ToTensor, Normalize

In [ ]:
init_notebook_mode(True)

In [ ]:
def parse(path):
    result = match(r'^.*/(\w+)\.(\w+)\.webp$', path)
    return ('unknown', 'unknown') if not result else result.groups()    

In [ ]:
def d3(matrix):
    return stack((matrix, matrix, matrix), 2)

In [ ]:
url = '../data/auge/images/d379f0f7ed01b7290a3a905e9daba1d1276ef7d3.comfort.webp'
gray = imread(url, IMREAD_GRAYSCALE)
mask = copy(gray)
mask[341:443, 125:272] = 255
mask[:341, :] = 0
mask[443:, :] = 0
mask[:, :125] = 0
mask[:, 272:] = 0
for x in range(125, 165):
    mask[:382 + 126 - x, x] = 0
for x in range(210, 272):
    mask[:x + 130, x] = 0
name, label = parse(url)
iplot(
    make_subplots(cols=2, horizontal_spacing=0.03)
    .add_trace(Image(z=d3(gray), name=''), 1, 1)
    .add_trace(Image(z=d3(mask), name=''), 1, 2)
    .update_layout(title=f'{name} - {label}', margin={'t': 40, 'r': 0, 'b': 0, 'l': 0}, height=380)
    .update_xaxes(showticklabels=False)
    .update_yaxes(showticklabels=False)
)
imwrite('../data/auge/masks/olymp.jpeg', mask)
pass

In [ ]:
url = '../data/auge/images/b7bb98d1f53cbc8614fad2e5926975a13904e135.construction.webp'
bgr = imread(url, IMREAD_COLOR)
name, label = parse(url)
iplot(
    make_subplots(cols=2, horizontal_spacing=0.03)
    .add_trace(Image(z=cvtColor(bgr, COLOR_BGR2RGB), name=''), 1, 1)
    .add_trace(
        Image(
            z=cvtColor(
                inpaint(bgr, imread('../data/auge/masks/domria.jpeg', IMREAD_GRAYSCALE), 4, INPAINT_NS),
                COLOR_BGR2RGB
            ),
            name=''
        ),
        1,
        2
    )
    .update_layout(title=f'{name} - {label}', margin={'t': 40, 'r': 0, 'b': 0, 'l': 0}, height=380)
    .update_xaxes(showticklabels=False)
    .update_yaxes(showticklabels=False)
)

In [ ]:
labels = ['unknown', 'luxury', 'comfort', 'junk', 'construction', 'obsolete', 'outline', 'novelty', 'view']
mappings = {l: i for i, l in enumerate(labels)}

In [ ]:
images = DataFrame({'path': glob('../data/auge/images/*.webp')})
images['label'] = images['path'].apply(lambda p: mappings[parse(p)[1]])

In [ ]:
counts = images['label'].value_counts()
iplot(
    Figure()
    .add_trace(Pie(labels=[labels[i] for i in counts.index], values=counts.values))
    .update_layout(margin={'t': 30, 'r': 0, 'b': 0, 'l': 0})
)

In [ ]:
class AugeDataset(Dataset):
    def __init__(self, data):
        self._data = data.values
        self._transforms = Compose(
            [ToTensor(), Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225], True)]
        )

    def __len__(self):
        return len(self._data)

    def __getitem__(self, index):
        path, label = self._data[index]
        return self._transforms(cvtColor(imread(path, IMREAD_COLOR), COLOR_BGR2RGB)), label

In [ ]:
training_images, testing_images = train_test_split(images, test_size=0.2, stratify=images['label'])
training_images, validation_images = train_test_split(
    training_images,
    test_size=0.2,
    stratify=training_images['label']
)
batch_size = 4
training_loader = DataLoader(AugeDataset(training_images), batch_size, True)
validation_loader = DataLoader(AugeDataset(validation_images), batch_size)
testing_loader = DataLoader(AugeDataset(testing_images), batch_size)